In [1]:
import keras_nlp
import json
samples_train = []
with open('/kaggle/input/quality-v1-0-1/QuALITY.v1.0.1.htmlstripped.train', 'r') as file:
    for line in file:
        samples_train.append(json.loads(line))
        
samples_dev = []
with open('/kaggle/input/quality-v1-0-1/QuALITY.v1.0.1.htmlstripped.dev', 'r') as file:
    for line in file:
        samples_dev.append(json.loads(line))

In [2]:
tokenizer = keras_nlp.models.GemmaTokenizer.from_preset("gemma2_instruct_27b_en")

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [3]:
import numpy as np
lens_train=[]
for sample in samples_train:
    lens_train.append(tokenizer.tokenize(sample['article']).shape[0])
lens_train=np.array(lens_train)

lens_dev=[]
for sample in samples_dev:
    lens_dev.append(tokenizer.tokenize(sample['article']).shape[0])
lens_dev=np.array(lens_dev)

In [4]:
samples_short_train=[]
for i in range(len(lens_train)):
    if(lens_train[i]<=4000):
        samples_short_train.append(samples_train[i])
        
samples_short_dev=[]
for i in range(len(lens_dev)):
    if(lens_dev[i]<=4000):
        samples_short_dev.append(samples_dev[i])

In [5]:
print(len(samples_short_train))
print(len(samples_short_dev))

72
56


In [6]:
import random

def get_data_hard(samples_short):
    random.seed(42)

    selected_data_hard = []

    for text_index, text in enumerate(samples_short):
        for question_index, question in enumerate(text['questions']):
            
            best_question = question
            best_question_index = question_index

            if best_question['gold_label'] - 1 == best_question['writer_label'] - 1 and best_question['difficult'] == 1:
                correct_index = best_question['gold_label'] - 1  # Subtract 1 to adjust for 1-based index
                correct_option = best_question['options'][correct_index]

                # Choose a wrong answer randomly
                wrong_indices = [i for i in range(len(best_question['options'])) if i != correct_index]
                wrong_index = random.choice(wrong_indices)
                wrong_option = best_question['options'][wrong_index]

                selected_data_hard.append({
                    'text_index': text_index,  # Track the text index
                    'article' : samples_short[text_index]['article'],
                    'question_index': best_question_index,
                    'question': best_question['question'],
                    'correct_answer_index': correct_index,
                    'correct_answer': correct_option,
                    'wrong_answer_index': wrong_index,
                    'wrong_answer': wrong_option
                })

    # Verify the correctness of the selected data
    for selected in selected_data_hard:
        original_question = samples_short[selected['text_index']]['questions'][selected['question_index']]
        assert selected['correct_answer'] == original_question['options'][selected['correct_answer_index']], "Correct answer doesn't match"
        assert selected['wrong_answer'] == original_question['options'][selected['wrong_answer_index']], "Wrong answer doesn't match"

    print("All selected data entries are correct!")
    return selected_data_hard

In [7]:
import random

def get_data_easy(samples_short):
    random.seed(42)

    selected_data_easy = []

    for text_index, text in enumerate(samples_short):
        for question_index, question in enumerate(text['questions']):
            
            best_question = question
            best_question_index = question_index

            if best_question['gold_label'] - 1 == best_question['writer_label'] - 1 and best_question['difficult'] == 0:
                correct_index = best_question['gold_label'] - 1  # Subtract 1 to adjust for 1-based index
                correct_option = best_question['options'][correct_index]

                # Choose a wrong answer randomly
                wrong_indices = [i for i in range(len(best_question['options'])) if i != correct_index]
                wrong_index = random.choice(wrong_indices)
                wrong_option = best_question['options'][wrong_index]

                selected_data_easy.append({
                    'text_index': text_index,  # Track the text index
                    'article' : samples_short[text_index]['article'],
                    'question_index': best_question_index,
                    'question': best_question['question'],
                    'correct_answer_index': correct_index,
                    'correct_answer': correct_option,
                    'wrong_answer_index': wrong_index,
                    'wrong_answer': wrong_option
                })

    # Verify the correctness of the selected data
    for selected in selected_data_easy:
        original_question = samples_short[selected['text_index']]['questions'][selected['question_index']]
        assert selected['correct_answer'] == original_question['options'][selected['correct_answer_index']], "Correct answer doesn't match"
        assert selected['wrong_answer'] == original_question['options'][selected['wrong_answer_index']], "Wrong answer doesn't match"

    print("All selected data entries are correct!")
    return selected_data_easy

In [8]:
selected_data_easy_train=get_data_easy(samples_short_train)
selected_data_hard_train=get_data_hard(samples_short_train)

selected_data_easy_dev=get_data_easy(samples_short_dev)
selected_data_hard_dev=get_data_hard(samples_short_dev)

All selected data entries are correct!
All selected data entries are correct!
All selected data entries are correct!
All selected data entries are correct!


In [9]:
print(len(selected_data_easy_train))
print(selected_data_easy_train[0]['question'])
print(len(selected_data_hard_train))
print(selected_data_hard_train[0]['question'])

print(len(selected_data_easy_dev))
print(selected_data_easy_dev[0]['question'])
print(len(selected_data_hard_dev))
print(selected_data_hard_dev[0]['question'])

339
What is the writer's view of Mehta's works?
222
Why is it suspected that William Shawn blushed at Green's remark? 
322
Why did the press not report on JFK's adultery?
177
Why did people say the story about Clinton hiding under a blanket to meet a woman was untrue?


In [10]:
# Extract all articles from the training and development datasets
train_articles = {entry['article'] for entry in selected_data_easy_train + selected_data_hard_train}
dev_articles = {entry['article'] for entry in selected_data_easy_dev + selected_data_hard_dev}

# Find the intersection of the two sets to see if there are any common articles
common_articles = train_articles.intersection(dev_articles)

# Print the result
if common_articles:
    print(f"There are {len(common_articles)} articles in common between training and development datasets.")
else:
    print("There are no articles in common between training and development datasets.")

There are no articles in common between training and development datasets.


In [11]:
import pandas as pd

# Function to convert a dataset to a DataFrame and save it to CSV
def save_dataset_to_csv(dataset, filename):
    df = pd.DataFrame(dataset)
    df.to_csv(filename, index=False)

# Save each dataset to a CSV file
save_dataset_to_csv(selected_data_easy_train, 'selected_data_easy_train.csv')
save_dataset_to_csv(selected_data_hard_train, 'selected_data_hard_train.csv')
save_dataset_to_csv(selected_data_easy_dev, 'selected_data_easy_dev.csv')
save_dataset_to_csv(selected_data_hard_dev, 'selected_data_hard_dev.csv')

print("Datasets saved to CSV files.")

Datasets saved to CSV files.


In [12]:
import pandas as pd

# Function to load a CSV and compare with the original dataset
def compare_csv_with_original(dataset, csv_filename):
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_filename)
    
    # Convert the original dataset to a DataFrame
    original_df = pd.DataFrame(dataset)
    
    # Compare the "article" and "question" columns
    articles_equal = original_df['article'].equals(df['article'])
    questions_equal = original_df['question'].equals(df['question'])
    
    # Print the comparison results
    if articles_equal and questions_equal:
        print(f"The CSV file {csv_filename} is equal to the original dataset.")
    else:
        print(f"The CSV file {csv_filename} differs from the original dataset.")

# Compare each dataset with its corresponding CSV file
compare_csv_with_original(selected_data_easy_train, 'selected_data_easy_train.csv')
compare_csv_with_original(selected_data_hard_train, 'selected_data_hard_train.csv')
compare_csv_with_original(selected_data_easy_dev, 'selected_data_easy_dev.csv')
compare_csv_with_original(selected_data_hard_dev, 'selected_data_hard_dev.csv')


The CSV file selected_data_easy_train.csv is equal to the original dataset.
The CSV file selected_data_hard_train.csv is equal to the original dataset.
The CSV file selected_data_easy_dev.csv is equal to the original dataset.
The CSV file selected_data_hard_dev.csv is equal to the original dataset.
